In [1]:
import sys
sys.path.append('../scripts')

from os import path
from utils import cleanCmpnyNm, getGrpCmpnyDict
import numpy as np
import pandas as pd

In [17]:
# merge cmpnySumry
lCmpnySumry = []
for year in range(2020, 2022):
    df = pd.read_excel(f'../data/cmpnySumry/cmpnySumry{year}Kor.xlsx')
    lCmpnySumry.append(df)
    print(len(list(df.columns)))
dfCmpnySumry = pd.concat(lCmpnySumry, axis=0)
dfCmpnySumry.head()

23
23


,공개년월,회계년,기업집단명,소속회사명,대표자,설립일,계열편입일,업종코드,영위업종,종업원수,...,자산총액,부채총액,자본총액,자본금,매출액,당기순이익,구분,법인등록번호,year2,rcmgCode
0,202005,2019,넥슨,(유)가승개발,서춘식,2016/01/21,2016/01/21,R91,스포츠 및 오락관련 서비스업,3,...,"15,208","6,064","9,144",100,0,-465,일반회사,1345140003989,2020,1.802408e+09
1,202005,2019,지에스,(유)가승개발,서춘식,2016/01/21,2016/04/01,R91,스포츠 및 오락관련 서비스업,3,...,"15,208","6,064","9,144",100,NaN,-465,일반회사,1345140003989,2020,1.802408e+09
2,202005,2019,카카오,(유)매디슨카운티의다리,이명한,2016/11/16,2020/02/03,R90,"창작, 예술 및 여가관련 서비스업",0,...,11,28,-17,10,3,2,일반회사,1101140192605,2020,1.803409e+09
3,202005,2019,엘에스,(유)빌드윈문경태양광발전,김선철,2018/12/11,2019/02/01,D35,"전기, 가스, 증기 및 공기 조절 공급업",1,...,331,0,331,331,0,0,일반회사,1754140000582,2020,2.102105e+09
4,202005,2019,엘에스,(유)빌드윈인동태양광발전,김선철,2018/12/11,2019/02/01,D35,"전기, 가스, 증기 및 공기 조절 공급업",1,...,"4,883","4,140",743,756,0,-13,일반회사,1760140002014,2020,2.102105e+09


In [18]:
dfCmpnySumry.shape

(4896, 23)

In [19]:
dfCmpnySumry.columns

Index(['공개년월', '회계년', '기업집단명', '소속회사명', '대표자', '설립일', '계열편입일', '업종코드', '영위업종',
       '종업원수', '결산기일', '결산주총일', '기업공개일', '자산총액', '부채총액', '자본총액', '자본금', '매출액',
       '당기순이익', '구분', '법인등록번호', 'year2', 'rcmgCode'],
      dtype='object')

In [20]:
len(dfCmpnySumry.columns)

23

In [22]:
dfCmpnySumry.columns = ['publicYm', 'fisclYr', 'unityGrupNm', 'entrprsNm', 'rprsntvNm', 'fondDe',\
                          'grinil', 'indutyCode', 'indutyNm', 'ordtmEmplyCo', 'stacntDudt', 'beforeBsnsStacntDe',\
                          'entrprsOthbcDe', 'assetsTotamt', 'debtTotamt', 'caplTotamt', 'caplAmount', 'selngAmount', 'thstrmNtpfAmount',\
                          'reconstrctur', 'jurirno', 'year2', 'rcmgCode']

In [23]:
dfCmpnySumry['entrprsNm'] = cleanCmpnyNm(dfCmpnySumry['entrprsNm'])
dfCmpnySumry['entrprsNm'].nunique()

2833

In [8]:
dfCmpnySumry.to_excel(f'../cmpnySumry2001-2021Eng.xlsx')

In [10]:
dfCmpnySumry.columns

Index(['publicYm', 'fisclYr', 'unityGrupNm', 'entrprsNm', 'rprsntvNm',
       'fondDe', 'grinil', 'indutyCode', 'indutyNm', 'ordtmEmplyCo',
       'stacntDudt', 'beforeBsnsStacntDe', 'entrprsOthbcDe', 'assetsTotamt',
       'debtTotamt', 'caplTotamt', 'caplAmount', 'selngAmount',
       'thstrmNtpfAmount', 'reconstrctur', 'jurirno', 'year2', 'rcmgCode'],
      dtype='object')

In [ ]:
dfCmpnySumry.columns = ['publicYm', 'fisclYr', 'unityGrupNm', 'entrprsNm', 'rprsntvNm', 'fondDe',\
                        'grinil', 'indutyCode', 'indutyNm', 'ordtmEmplyCo', 'stacntDudt', 'beforeBsnsStacntDe',\
                        'entrprsOthbcDe', 'assetsTotamt', 'debtTotamt', 'caplTotamt', 'caplAmount', 'selngAmount', 'thstrmNtpfAmount',\
                        'reconstrctur', 'jurirno', 'year2']

# 2019